In [2]:
from core.meta import meta_interface

%matplotlib inline

In [3]:
d = meta_interface(project_config_path="test_data/project_config.yaml")
# manually add recording_configs
d.add_recording_config(
    "test_data/Server_structure/Calibrations/220826/recording_config.yaml"
)
d.initialize_meta_config()
# switch recordings to other Recording_days -> calibration_directroy

In [4]:
d.create_recordings()
# update fps, video_filepath, target_fps

In [ ]:
d.synchronize_recordings()  # test_mode: bool, doesnt overwrite synchronized and analysed videos
# exclude recordingvideos based on synchronization

In [10]:
d.create_calibrations()
# update fps, video_filepath, target_fps

In [ ]:
d.synchronize_calibrations()  # test_mode: bool, doesnt overwrite synchronized videos
# exclude calibrationvideos based on synchronization, change markerdetection_filepaths etc.

In [ ]:
d.calibrate()
# exclude calibrations based on calibration log and repeat calibration (not yet implemented) 

In [18]:
d.triangulate_recordings()

In [16]:
d.export_meta_to_yaml(filename="test_data/meta_test.yaml")

In [17]:
d.load_meta_from_yaml(filename="test_data/meta_test.yaml")

In [ ]:
from core.triangulation_calibration_module import (
    Calibration,
    Triangulation_Positions,
    Triangulation_Recordings,
)

In [ ]:
a = Calibration(
    calibration_directory="test_data/Server_structure/Calibrations/220826/",
    project_config_filepath="test_data/project_config.yaml",
    recording_config_filepath="test_data/Server_structure/Calibrations/220826/recording_config.yaml",
    output_directory="test_data/Server_structure/Calibrations/220826/",
)

In [ ]:
a.run_synchronization()

In [ ]:
a.run_calibration(verbose=True)

In [ ]:
b = Triangulation_Recordings(
    recording_directory="test_data/Server_structure/VGlut2-flp/September2022/196_F7-27/220826_OTT/",
    calibration_directory="test_data/Server_structure/Calibrations/220826/",
    recording_config_filepath="test_data/Server_structure/Calibrations/220826/recording_config.yaml",
    project_config_filepath="test_data/project_config.yaml",
    output_directory="test_data/Server_structure/VGlut2-flp/September2022/196_F7-27/220826_OTT/",
)

In [ ]:
b.run_synchronization()

In [ ]:
b.run_triangulation(
    calibration_toml_filepath="test_data/Server_structure/Calibrations/220826/calibration_220826.toml"
)

In [ ]:
c = Triangulation_Positions(
    positions_directory="test_data/Server_structure/Calibrations/220826/",
    calibration_directory="test_data/Server_structure/Calibrations/220826/",
    recording_config_filepath="test_data/Server_structure/Calibrations/220826/recording_config.yaml",
    project_config_filepath="test_data/project_config.yaml",
    output_directory="test_data/Server_structure/Calibrations/220826/",
)

In [ ]:
c.get_marker_predictions()

In [ ]:
c.run_triangulation(
    calibration_toml_filepath="test_data/Server_structure/Calibrations/220826/calibration_220826.toml"
)

In [ ]:
# special dependencies

In [ ]:
"""
pip install aniposelib
pip install ffmpeg
pip install ffmpeg-python
pip install imageio-ffmpeg
jupyter lab
Tkinter
"""